This script is the same as the jupyter notebook provided by you to train the embeddings model on hindi dataset.
It works for the Bangali dataset and the only difference is that I choose around 5000 samples of the dataset half of which have label 0 and the other half have label 1 so that the statistics of the dataset match that of the hindi dataset (since we want to do fine tuning from the model trained on hindi dataset). The rest of the script is the same. Please refer to Task1_Word_Embeddings.ipynp for comments on different parts of the code.

In [ ]:

import torch
import csv
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import pickle
import random
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")


# Imports
import numpy as np
import csv
from utils import *


data = open('data/bengali_hatespeech.csv')
data = csv.reader(data, delimiter='\t')
tweets = [line for line in data]

# ## 1.2 Data preparation (0.5 + 0.5 points)


# TODO: implement!
stopwords = [line[:-2] for line in open('bangali_stopwords')]

def valid(word):
    if word.startswith('@') or word.startswith('(@') or word.startswith('.@'):
        return False
    if word in [',', ';', ':', '.', '!', '?', '\'', '\""', '-', '_', '/', '(', ')', '[', ']', '...', '*']:
        return False
    if word in stopwords:
        return False
    #     if word in english_stopwords:
    #         return False
    if word.startswith('http'):
        return False
    return True


new_tweets = []
labels = []
indices = np.arange(len(tweets))
random.shuffle(indices)
num_0 = 0
num_1 = 0
# tweets = tweets[indices]
for i in indices:
    if num_0>2500 and num_1>2500:
        break
    # print(tweets[i][0].split(',')[1])
    tweet = tweets[i][0].split(',')
    if tweet[1] not in ['0','1']:
        continue
    if i==0:
        continue
    new_sentence = []
    # print(tweet)
    if tweet[1] == '0':
        num_0 += 1
        if num_0>2500:
            continue
    if tweet[1] == '1':
        num_1 += 1
        if num_1>2500:
            continue

    labels.append(int(tweet[1]))
    for word in tweet[0].split():
#         print(word)
        if valid(word):
            cnt = 0
            while(word[cnt]==' '):
                cnt += 1
            word = word[cnt:]
            new_sentence.append(word.lower())
#                 print(word)
    new_tweets.append(new_sentence)

word_dict = {}
all_words = []
for tweet in new_tweets:
    for word in tweet:
        if not word in word_dict.keys():
            all_words.append(word)
            word_dict[word] = 1
        else:
            word_dict[word] += 1
V = all_words


# * Then, write a function ```word_to_one_hot``` that returns a one-hot encoding of an arbitrary word in the vocabulary. The size of the one-hot encoding should be ```len(v)```.

# In[5]:


# TODO: implement!
def word_to_one_hot(word):
    one_hot = np.zeros(len(V))
    for i, w in enumerate(V):
        if word == w:
            one_hot[i] = 1
            break
    assert (np.sum(one_hot) == 1)
    return one_hot


import math

Z = word_dict


# print(Z)
def sampling_prob(word):
    return (math.sqrt(Z[word] / 0.001) + 1) * (0.001 / Z[word])


def get_target_context(sentence):
    context_array = []
    for i, word in enumerate(sentence):
        for w in range(-window_size, window_size + 1):
            context_idx = i + w
            if context_idx < 0 or context_idx >= len(sentence) or context_idx == i:
                continue
            keep = np.random.choice(2, 1, p=[1.0 - sampling_prob(word), sampling_prob(word)])
            if keep:
                context_array.append((word, sentence[context_idx]))
    return context_array


# Set hyperparameters
window_size = 5
embedding_size = 300

# More hyperparameters
learning_rate = 0.007
epochs = 50


# Create model
import torch.nn as nn


class Word2Vec(nn.Module):
    def __init__(self):
        super().__init__()
        self.hidden_1 = nn.Linear(len(V), embedding_size)
        self.hidden_2 = nn.Linear(embedding_size, len(V))
        self.logsoftmax = nn.LogSoftmax()

    def forward(self, one_hot):
        out = self.hidden_1(one_hot)
        out = self.hidden_2(out)
        out = self.logsoftmax(out)
        return out


# # 1.8 Loss function and optimizer (0.5 points)


# Define optimizer and loss
import torch

my_model = Word2Vec().to(device)
optimizer = torch.optim.Adam(my_model.parameters(), lr=learning_rate)
criterion = nn.NLLLoss()



# Define train procedure

# load initial weights
def train():
    print("Training started")
    X = []
    Y = []
    for sentence in new_tweets:
        for tuples in get_target_context(sentence):
            X.append(word_to_one_hot(tuples[0]))
            Y.append(int(np.where(word_to_one_hot(tuples[1]) == 1)[0][0]))


    num_data = len(X)
    num_train = int(0.8 * num_data)
    X_train = torch.tensor(X[:num_train])
    Y_train = torch.tensor(Y[:num_train])


    batch_size = 64
    num_batches = int(len(X_train) / batch_size)
    for epoch in range(epochs):
        print(epoch)
        epoch_loss = 0
        for i in range(num_batches):
            X_batch = X_train[i * batch_size:(i + 1) * batch_size].float().to(device)
            Y_batch = Y_train[i * batch_size:(i + 1) * batch_size].to(device)
            # print(X_batch)
            # print(Y_batch)
            optimizer.zero_grad()
            loss = criterion(my_model(X_batch), Y_batch)
            loss.backward()
            optimizer.step()
            epoch_loss += float(loss)
        print('train loss : ', float(epoch_loss / num_batches))



train()

print("Training finished")


for param_tensor in my_model.state_dict():
    print(param_tensor, "\t", my_model.state_dict()[param_tensor].size())
torch.save(my_model.state_dict(), 'bangali_embeddings_model')

data = {}
data['tweets'] = new_tweets
data['Y'] = np.array(labels)
data['all_words'] = np.array(V)
save_data(data, 'bangali_tweets')






/home/nastaran/anaconda3/lib/python3.8/site-packages/torch/cuda/__init__.py:52: UserWarning: CUDA initialization: Found no NVIDIA driver on your system. Please check that you have an NVIDIA GPU and installed a driver from http://www.nvidia.com/Download/index.aspx (Triggered internally at  /opt/conda/conda-bld/pytorch_1607369981906/work/c10/cuda/CUDAFunctions.cpp:100.)
  return torch._C._cuda_getDeviceCount() > 0


Training started
0


<ipython-input-2-a895499a6d83>:154: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  out = self.logsoftmax(out)


train loss :  9.166740721550541
1
train loss :  7.3705848403598955
2
train loss :  6.718367562777754
3
train loss :  4.915612483370131
4
train loss :  2.999387532040693
5
train loss :  1.6871720630189646
6
train loss :  0.8961927515009175
7
train loss :  0.7458769342173701
8
train loss :  0.7260955395042032
9
train loss :  0.7102453933245894
10
train loss :  0.6657487331093221
11
train loss :  0.6286886649719183
12
train loss :  0.6061615451522495
13
train loss :  0.5956463528716046
14
train loss :  0.586918209341989
15
train loss :  0.580484431290972
16
train loss :  0.5727338993894882
17
train loss :  0.5658252329930015
18
train loss :  0.5593273777892624
19
train loss :  0.554622512364733
20
train loss :  0.5548610756362694
21
train loss :  0.5482034324735835
22
train loss :  0.5457333097423333
23
train loss :  0.542739461729492
24
train loss :  0.5422034060609513
25
train loss :  0.5377405018046282
26
train loss :  0.5366553042245947
27
